<a href="https://colab.research.google.com/github/AmiriShavaki/KG-based-RAG-for-Multi-hop-QA/blob/main/src/Construct_Knowledge_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install requirements

In [ ]:
!pip install langchain neo4j openai wikipedia tiktoken langchain_openai langchain-community

## Connect to the database

In [ ]:
from langchain.graphs import Neo4jGraph

url = input("Enter your Neo4j URL: ")

username = input("Enter your Neo4j username: ")
password = input("Enter your Neo4j password: ")

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

## Delete the graph if needed (uncomment before run)

In [ ]:
# graph.query("MATCH (n) DETACH DELETE n")

[]

## Knowledge graph data structure definition

In [ ]:
from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

## Utility mapping functions

In [ ]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

## The prompt and LLM configuration

In [ ]:
import os
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = password = input("Enter your OpenAI API Key: ")
llm = ChatOpenAI(model="gpt-4o", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

## extract_and_store_graph function definition

In [ ]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:

    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke(document.page_content)['function']

    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )

    graph.add_graph_documents([graph_document])

## Load dataset documents

In [ ]:
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
import csv
from collections import defaultdict

def read_documents_from_csv(filename, document_class=Document):
  """
  Reads page_contents from a CSV file and creates a list of documents, ensuring unique entries.

  Args:
      filename: Path to the CSV file.
      document_class: Class used to create document objects (defaults to Document).

  Returns:
      A list of Document objects with unique page_content.
  """
  unique_contents = defaultdict(int)
  documents = []

  with open(filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for row in reader:
      text = row[0]
      if unique_contents[text] == 0:
        documents.append(document_class(page_content=text))
      unique_contents[text] += 1

  return documents

raw_documents = read_documents_from_csv('shuffled_dataset.csv')

text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)

documents = text_splitter.split_documents(raw_documents)

## Run extract_and_store_graph on documents

In [ ]:
from tqdm import tqdm

allowed_nodes = ["Club", "Player", "Trophy", "Stadium", "Number", "Document", "Date", "Aircraft", "Animal", "Diet", "Airline", "Programming Language", "Company", "Novel", "Writer", "Prize", "Theme", "Building", "Material"]
allowed_relations = ["Won", "Signed", "Transferred to", "Joined", "Shirt colour", "Home stadium of", "Had jersey number of", "Located in", "Where was signed", "Has captained", "Had role of", "Architectural style of", "Largest operator in", "Used by", "Operates over", "Diet consists of", "Largest land animal found in", "Main theme of", "Tallest in", "Used in"]

for i, d in tqdm(enumerate(documents), total=len(documents)):
    try:
        extract_and_store_graph(d, allowed_nodes, allowed_relations)
    except:
        print("Error:", d)

100%|██████████| 498/498 [15:17<00:00,  1.84s/it]


## Write all relations to a text file

In [ ]:
data = graph.query("Match (n)-[r]->(m) Return n,r,m")

with open('relations.txt', 'w', encoding='utf-8') as file:
    for item in data:
        try:
            n = item['n']['name']
            r = item['r'][1]
            m = item['m']['name']
            file.write(f"{n}\t{r}\t{m}\n")
        except:
            print("Error on ", item)

Error on  {'n': {'name': 'Megan Rapinoe', 'id': 'Megan Rapinoe'}, 'r': ({'name': 'Megan Rapinoe', 'id': 'Megan Rapinoe'}, 'HAD_ROLE_OF', {'id': 'Left-Sided Wide Forward'}), 'm': {'id': 'Left-Sided Wide Forward'}}
Error on  {'n': {'name': 'Megan Rapinoe', 'id': 'Megan Rapinoe'}, 'r': ({'name': 'Megan Rapinoe', 'id': 'Megan Rapinoe'}, 'HAS_CAPTAINED', {'id': 'Team'}), 'm': {'id': 'Team'}}
Error on  {'n': {'id': 'Woollymammoth'}, 'r': ({'id': 'Woollymammoth'}, 'LARGEST_LAND_ANIMAL_FOUND_IN', {'id': 'Siberia'}), 'm': {'id': 'Siberia'}}
Error on  {'n': {'id': 'Woollymammoth'}, 'r': ({'id': 'Woollymammoth'}, 'LARGEST_LAND_ANIMAL_FOUND_IN', {'id': 'Northamerica'}), 'm': {'id': 'Northamerica'}}
Error on  {'n': {'name': 'Mayflower', 'id': 'Mayflower'}, 'r': ({'name': 'Mayflower', 'id': 'Mayflower'}, 'OPERATES_OVER', {'id': 'North America'}), 'm': {'id': 'North America'}}
Error on  {'n': {'id': 'Tyrannosaurus Rex'}, 'r': ({'id': 'Tyrannosaurus Rex'}, 'DISCOVERED_IN', {'id': 'North America'}), 'm